<a href="https://colab.research.google.com/github/chetnakbishnoi/Colab/blob/main/Another_copy_of_Final_UNET%26RESNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install geotile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.2 MB/s eta 0:00:00


In [ ]:
pip install fiona

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 98.9 MB/s eta 0:00:00


In [ ]:
import fiona

In [ ]:
from geotile import GeoTile
gt = GeoTile('/content/drive/MyDrive/iirs/Delhi_NCR_Buildings.tif')
gt.rasterization('/content/drive/MyDrive/utm_converted/utm_again.shp', out_path='/content/drive/MyDrive/iirs/Output_Delhi_NCR_Buildings.tif')

In [ ]:
gt_img = GeoTile('/content/drive/MyDrive/iirs/Delhi_NCR_Buildings.tif')
gt_img.generate_tiles(output_folder='/content/drive/MyDrive/tiles_of_tiff_delhi_ncr')


In [ ]:
# create the tiles of the raster mask
gt_mask = GeoTile('/content/drive/MyDrive/iirs/Output_Delhi_NCR_Buildings.tif')
gt_mask.generate_tiles(output_folder='/content/drive/MyDrive/raster_mask_tiles')

In [ ]:
import numpy as np
import os
from skimage import io
from skimage.exposure import rescale_intensity

# Directories for the satellite imagery and masks
satellite_dir = '/content/drive/MyDrive/tiles_of_tiff_delhi_ncr'
mask_dir = '/content/drive/MyDrive/raster_mask_tiles'
output_sat_dir = '/content/drive/MyDrive/processed_tiles_of_tiff_delhi_ncr'
output_mask_dir = '/content/drive/MyDrive/processed_raster_mask_tiles'

# Function to preprocess satellite imagery
def preprocess_satellite_image(image):
    # Replace NaN values with zeros
    image = np.nan_to_num(image, nan=0)

    # Normalize to [0, 1] range
    image = rescale_intensity(image, in_range=(image.min(), image.max()), out_range=(0, 1))

    return image

# Function to ensure mask is binary
def preprocess_mask(mask):
    # Clip values to ensure mask contains only 0 and 1
    mask = np.clip(mask, 0, 1)

    return mask

# Preprocess all satellite images and masks
for filename in os.listdir(satellite_dir):
    if filename.endswith(".tif"):
        # File paths
        satellite_path = os.path.join(satellite_dir, filename)
        mask_path = os.path.join(mask_dir, filename)  # Assuming the mask filenames match the satellite images

        # Load the satellite image and corresponding mask
        satellite_img = io.imread(satellite_path)
        mask_img = io.imread(mask_path)

        # Preprocess satellite image and mask
        preprocessed_satellite = preprocess_satellite_image(satellite_img)
        preprocessed_mask = preprocess_mask(mask_img)

        # Save the preprocessed satellite image and mask
        output_sat_path = os.path.join(output_sat_dir, filename)
        output_mask_path = os.path.join(output_mask_dir, filename)
        io.imsave(output_sat_path, (preprocessed_satellite * 255).astype(np.uint8))  # Save satellite image
        io.imsave(output_mask_path, preprocessed_mask.astype(np.uint8))  # Save mask

print("Preprocessing complete for all tiles.")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/tiles_of_tiff_delhi_ncr'

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation, Add,
    ZeroPadding2D, MaxPooling2D, AveragePooling2D, Dropout,
    UpSampling2D
)
from tensorflow.keras.initializers import glorot_uniform

# Define the ResNet50 architecture
def identity_block(X, f, filters, stage, block, l2_reg=0.01):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same',
               name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Add shortcut value to main path
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, stage, block, s=2, l2_reg=0.01):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(s, s), name=conv_name_base + '2a', padding='valid',
               kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(F2, (f, f), strides=(1, 1), name=conv_name_base + '2b', padding='same',
               kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1), strides=(1, 1), name=conv_name_base + '2c', padding='valid',
               kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Shortcut path
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s), name=conv_name_base + '1', padding='valid',
                        kernel_initializer=glorot_uniform(seed=0),
                        kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Add shortcut value to main path
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)

    return X

def ResNet50(input_shape=(256, 256, 3), classes=1):
    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')

    # Average Pooling
    X = AveragePooling2D(pool_size=(2, 2))(X)

    # Instead of Flatten, directly add the output layer
    X = Conv2D(classes, (1, 1), activation='sigmoid', name='output')(X)

    # Upsample to the desired output size (adjust this as needed)
    X = UpSampling2D(size=(32, 32), interpolation='bilinear')(X)

    # Update the model creation
    model = tf.keras.models.Model(inputs=X_input, outputs=X)

    return model

# Compile the model
model = ResNet50(input_shape=(256, 256, 3), classes=1)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Use your data loading function and train


In [ ]:
import os
import cv2  # or another image processing library
import numpy as np

def load_full_dataset(image_folder, mask_folder):
    images = []
    masks = []

    # Get all image and mask file paths
    image_filenames = sorted(os.listdir(image_folder))
    mask_filenames = sorted(os.listdir(mask_folder))

    for img_file, mask_file in zip(image_filenames, mask_filenames):
        # Load images
        img_path = os.path.join(image_folder, img_file)
        mask_path = os.path.join(mask_folder, mask_file)

        # Read and preprocess the image and mask (resize, normalize, etc.)
        image = cv2.imread(img_path)  # Load image
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask in grayscale

        if image is not None and mask is not None:
            # Resize images to the required input shape (256, 256)
            image = cv2.resize(image, (256, 256))
            mask = cv2.resize(mask, (256, 256))

            # Normalize image data to [0, 1] range if needed
            image = image.astype('float32') / 255.0
            mask = mask.astype('float32') / 255.0  # For binary masks, you can also keep it 0-1

            # Append to lists
            images.append(image)
            masks.append(mask)

    return np.array(images), np.array(masks)


In [ ]:
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Compile the model
model = ResNet50(input_shape=(256, 256, 3), classes=1)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Load your entire dataset
all_images, all_masks = load_full_dataset(train_image_folder, train_mask_folder)

# Ensure the data is in the right shape
all_images = np.array(all_images)  # Shape: (num_samples, 256, 256, 3)
all_masks = np.array(all_masks)     # Shape: (num_samples, 256, 256, 1)

# Define your batch size
batch_size = 32  # Adjust this based on your system's memory

# Define the number of epochs
epochs = 10  # You can adjust this

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Train the model
model.fit(all_images, all_masks,
          epochs=epochs,
          batch_size=batch_size,
          validation_split=0.2,  # Use a validation split for validation data
          callbacks=[early_stopping, model_checkpoint])

# Optionally, save the final model after training
model.save('final_resnet50_model.h5')


Epoch 1/10
3/3 [==============================] - ETA: 0s - loss: 65.9503 - accuracy: 0.7362

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3/3 [==============================] - 16s 2s/step - loss: 65.9503 - accuracy: 0.7362 - val_loss: 62.2402 - val_accuracy: 0.9243
Epoch 2/10
3/3 [==============================] - 6s 2s/step - loss: 60.6560 - accuracy: 0.9286 - val_loss: 56.8020 - val_accuracy: 0.9213
Epoch 3/10
3/3 [==============================] - 6s 2s/step - loss: 55.1733 - accuracy: 0.9211 - val_loss: 51.2371 - val_accuracy: 0.9243
Epoch 4/10
3/3 [==============================] - 6s 2s/step - loss: 49.7287 - accuracy: 0.9252 - val_loss: 45.9297 - val_accuracy: 0.9243
Epoch 5/10
3/3 [==============================] - 6s 2s/step - loss: 44.5515 - accuracy: 0.9270 - val_loss: 40.9731 - val_accuracy: 0.9243
Epoch 6/10
3/3 [==============================] - 5s 2s/step - loss: 39.7001 - accuracy: 0.9302 - val_loss: 36.4092 - val_accuracy: 0.9243
Epoch 7/10
3/3 [==============================] - 5s 2s/step - loss: 35.2539 - accuracy: 0.9261 - val_loss: 32.2551 - val_accuracy: 0.9243
Epoch 8/10
3/3 [=====================

In [ ]:
#LOSS IS HIGH

In [ ]:
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, ZeroPadding2D, MaxPooling2D, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import glorot_uniform


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation, Add,
    ZeroPadding2D, MaxPooling2D, AveragePooling2D, Dropout,
    UpSampling2D
)
from tensorflow.keras.initializers import glorot_uniform

# Define the ResNet50 architecture
def identity_block(X, f, filters, stage, block, l2_reg=0.01):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same',
               name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Add shortcut value to main path
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, stage, block, s=2, l2_reg=0.01):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(s, s), name=conv_name_base + '2a', padding='valid',
               kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(F2, (f, f), strides=(1, 1), name=conv_name_base + '2b', padding='same',
               kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1), strides=(1, 1), name=conv_name_base + '2c', padding='valid',
               kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Shortcut path
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s), name=conv_name_base + '1', padding='valid',
                        kernel_initializer=glorot_uniform(seed=0),
                        kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Add shortcut value to main path
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)

    return X

def ResNet50(input_shape=(256, 256, 3), classes=1):
    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')

    # Average Pooling
    X = AveragePooling2D(pool_size=(2, 2))(X)

    # Add Dropout layer
    X = Dropout(0.5)(X)

    # Instead of Flatten, directly add the output layer
    X = Conv2D(classes, (1, 1), activation='sigmoid', name='output')(X)

    # Upsample to the desired output size (adjust this as needed)
    X = UpSampling2D(size=(32, 32), interpolation='bilinear')(X)

    # Update the model creation with a name
    model = tf.keras.models.Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

# Compile the model
model = ResNet50(input_shape=(256, 256, 3), classes=1)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Use your data loading function and train the model


In [ ]:
import os
import numpy as np
import cv2

def load_full_dataset(train_image_folder, train_mask_folder):
    images = []
    masks = []

    # List all image files in the image folder
    image_files = sorted(os.listdir(train_image_folder))
    mask_files = sorted(os.listdir(train_mask_folder))

    for img_file, mask_file in zip(image_files, mask_files):
        # Construct full file path
        img_path = os.path.join(train_image_folder, img_file)
        mask_path = os.path.join(train_mask_folder, mask_file)

        # Load the image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Unable to read image {img_path}")
            continue
        img = cv2.resize(img, (256, 256))  # Resize to match the model input

        # Load the mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        if mask is None:
            print(f"Warning: Unable to read mask {mask_path}")
            continue
        mask = cv2.resize(mask, (256, 256))  # Resize to match the model input

        # Append to lists
        images.append(img)
        masks.append(mask.reshape((256, 256, 1)))  # Add channel dimension to masks

    return np.array(images), np.array(masks)

# Specify your folder paths
train_image_folder = '/content/drive/MyDrive/iirs/small_data/processed_raster_mask_tiles'
train_mask_folder = '/content/drive/MyDrive/iirs/small_data/processed_raster_mask_tiles'

# Load your entire dataset
all_images, all_masks = load_full_dataset(train_image_folder, train_mask_folder)

# Ensure the data is in the right shape
print(f"Loaded images shape: {all_images.shape}")  # Should be (99, 256, 256, 3)
print(f"Loaded masks shape: {all_masks.shape}")    # Should be (99, 256, 256, 1)

# Normalize images if necessary
all_images = all_images.astype('float32') / 255.0
all_masks = all_masks.astype('float32') / 255.0  # Ensure masks are also in the right format

# Check the normalized data shapes
print(f"Normalized images shape: {all_images.shape}")
print(f"Normalized masks shape: {all_masks.shape}")


Loaded images shape: (99, 256, 256, 3)
Loaded masks shape: (99, 256, 256, 1)
Normalized images shape: (99, 256, 256, 3)
Normalized masks shape: (99, 256, 256, 1)


In [ ]:
model = ResNet50(input_shape=(256, 256, 3), classes=1)  # Update the input shape and class count as needed


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import random

seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# Split the dataset into training and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(
    all_images, all_masks, test_size=0.2, random_state=42
)

# Define the data generator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

epochs=50
# Define your batch size
batch_size = 32  # Adjust this based on your system's memory

# Fit the model
model.fit(datagen.flow(train_images, train_masks, batch_size=batch_size),
          validation_data=(val_images, val_masks),  # Use the separate validation set
          epochs=epochs,
          callbacks=[early_stopping, reduce_lr, model_checkpoint])


Epoch 1/50
3/3 [==============================] - ETA: 0s - loss: 66.5246 - accuracy: 0.4257

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3/3 [==============================] - 16s 2s/step - loss: 66.5246 - accuracy: 0.4257 - val_loss: 63.1843 - val_accuracy: 0.5546 - lr: 0.0010
Epoch 2/50
3/3 [==============================] - 6s 2s/step - loss: 61.0306 - accuracy: 0.9294 - val_loss: 58.4365 - val_accuracy: 0.7787 - lr: 0.0010
Epoch 3/50
3/3 [==============================] - 6s 2s/step - loss: 56.5772 - accuracy: 0.9252 - val_loss: 53.3285 - val_accuracy: 0.9317 - lr: 0.0010
Epoch 4/50
3/3 [==============================] - 6s 2s/step - loss: 51.6521 - accuracy: 0.9340 - val_loss: 48.4529 - val_accuracy: 0.9323 - lr: 0.0010
Epoch 5/50
3/3 [==============================] - 6s 2s/step - loss: 47.1814 - accuracy: 0.9318 - val_loss: 43.7973 - val_accuracy: 0.9323 - lr: 0.0010
Epoch 6/50
3/3 [==============================] - 6s 2s/step - loss: 42.5955 - accuracy: 0.9306 - val_loss: 39.4080 - val_accuracy: 0.9323 - lr: 0.0010
Epoch 7/50
3/3 [==============================] - 6s 2s/step - loss: 37.9908 - accuracy: 0.9339 - 

In [ ]:
# Save the model to a specific path
model.save('/content/drive/MyDrive/saved_model/my_resnet50_model_final')


In [ ]:
model.save_weights('/content/drive/MyDrive/saved_model/my_model_weights.h5')


RESNET+UNET
**bold text**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def ResNet_UNet(input_shape=(256, 256, 3)):
    inputs = layers.Input(shape=input_shape)

    # Stage 1: Initial Conv + ZeroPadding
    x = layers.ZeroPadding2D(padding=(3, 3))(inputs)
    x = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    skip1 = x  # Skip connection 1 (Shape: 128x128)

    # Stage 2: MaxPool + ResNet Block 1 (conv2_x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = res_block(x, [64, 64, 256], strides=1)  # conv2_x block
    skip2 = x  # Skip connection 2 (Shape: 64x64)

    # Stage 3: ResNet Block 2 (conv3_x)
    x = res_block(x, [128, 128, 512], strides=2)  # conv3_x block
    skip3 = x  # Skip connection 3 (Shape: 32x32)

    # Stage 4: ResNet Block 3 (conv4_x)
    x = res_block(x, [256, 256, 1024], strides=2)  # conv4_x block
    skip4 = x  # Skip connection 4 (Shape: 16x16)

    # Stage 5: ResNet Block 4 (conv5_x)
    x = res_block(x, [512, 512, 2048], strides=2)  # conv5_x block

    # Decoder Stage 1: Up-sample and concatenate with skip connection 4 (16x16)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Concatenate()([x, skip4])  # 16x16

    # Decoder Stage 2: Up-sample and concatenate with skip connection 3 (32x32)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Concatenate()([x, skip3])  # 32x32

    # Decoder Stage 3: Up-sample and concatenate with skip connection 2 (64x64)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Concatenate()([x, skip2])  # 64x64

    # Decoder Stage 4: Up-sample and concatenate with skip connection 1 (128x128)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Concatenate()([x, skip1])  # 128x128

    # Final Convolution to return to original resolution
    x = layers.Conv2D(1, (1, 1), activation='sigmoid')(x)

    model = Model(inputs, x)
    return model

def res_block(x, filters, strides=1):
    # Filters = [filter1, filter2, filter3]
    filter1, filter2, filter3 = filters

    # First branch
    shortcut = x
    x = layers.Conv2D(filter1, (1, 1), strides=strides)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filter2, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filter3, (1, 1))(x)
    x = layers.BatchNormalization()(x)

    # Shortcut connection
    shortcut = layers.Conv2D(filter3, (1, 1), strides=strides)(shortcut)
    shortcut = layers.BatchNormalization()(shortcut)

    # Adding shortcut and main path
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)

    return x

# Compile and build the model
model = ResNet_UNet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 zero_padding2d_4 (ZeroPadd  (None, 262, 262, 3)          0         ['input_5[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 64)         9472      ['zero_padding2d_4[0][0]']    
                                                                                                  
 batch_normalization (Batch  (None, 128, 128, 64)         256       ['conv2d[0][0]']        

In [ ]:
# Save the model in HDF5 format
model.save('my_resnet50_model.h5')


In [ ]:
pip install tf2onnx onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 100.9 MB/s eta 0:00:00


In [ ]:
import tf2onnx
import tensorflow as tf
import onnx  # Import the onnx module

# Load your TensorFlow model
model = tf.keras.models.load_model('my_resnet50_model.h5')

# Convert the TensorFlow model to ONNX format
onnx_model, _ = tf2onnx.convert.from_keras(model)

# Save the ONNX model
onnx.save_model(onnx_model, 'my_resnet50_model.onnx')



In [ ]:
pip install onnxruntime onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.3 MB/s eta 0:00:00


In [ ]:
!pip install onnx2pytorch



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.4 MB/s eta 0:00:00


In [ ]:
import onnx2pytorch


In [ ]:
import torch
import onnx
import onnx2pytorch

# Load the ONNX model
onnx_model = onnx.load('my_resnet50_model.onnx')

# Convert ONNX model to PyTorch model
pytorch_model = onnx2pytorch.ConvertModel(onnx_model)

# Now you can use pytorch_model as a regular PyTorch model


/usr/local/lib/python3.10/dist-packages/onnx2pytorch/convert/layer.py:30: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))
/usr/local/lib/python3.10/dist-packages/onnx2pytorch/convert/attribute.py:101: UserWarning: Pytorch's interpolate uses no coordinate_transformation_mode=tf_half_pixel_for_nn. Result might differ.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/onnx2pytorch/operations/resize.py:16: UserWarning: Pytorch's interpolate uses no exclude_outside. Result might differ.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/onnx2pytorch

In [ ]:
print(f'Train images shape: {train_images.shape}')
print(f'Train masks shape: {train_masks.shape}')
print(f'Validation images shape: {val_images.shape}')
print(f'Validation masks shape: {val_masks.shape}')

Train images shape: (1243, 256, 256, 3)
Train masks shape: (1243, 256, 256, 1)
Validation images shape: (311, 256, 256, 3)
Validation masks shape: (311, 256, 256, 1)


STEPS


In [ ]:
pip install onnx onnxruntime onnx2pytorch


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Define the backbone as 'resnet50'
backbone = 'resnet50'

class DecoderBlock(nn.Module):
    """
    U-Net's decoding module.
    Uses a 3x3 convolution followed by an upsampling layer.
    """
    def __init__(self, in_channels, mid_channels, out_channels, upsample_mode='pixelshuffle', BN_enable=True):
        super().__init__()
        self.in_channels = in_channels
        self.mid_channels = mid_channels
        self.out_channels = out_channels
        self.upsample_mode = upsample_mode
        self.BN_enable = BN_enable

        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=mid_channels, kernel_size=3, stride=1, padding=1, bias=False)

        if self.BN_enable:
            self.norm1 = nn.BatchNorm2d(mid_channels)
        self.relu1 = nn.ReLU(inplace=False)
        self.relu2 = nn.ReLU(inplace=False)

        if self.upsample_mode == 'deconv':
            self.upsample = nn.ConvTranspose2d(in_channels=mid_channels, out_channels=out_channels,
                                                kernel_size=3, stride=2, padding=1, output_padding=1, bias=False)
        elif self.upsample_mode == 'pixelshuffle':
            self.upsample = nn.PixelShuffle(upscale_factor=2)
        if self.BN_enable:
            self.norm2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        if self.BN_enable:
            x = self.norm1(x)
        x = self.relu1(x)
        x = self.upsample(x)
        if self.BN_enable:
            x = self.norm2(x)
        x = self.relu2(x)
        return x

class Resnet_Unet(nn.Module):
    """
    U-Net model using ResNet-50 as the backbone.
    """
    def __init__(self, BN_enable=True):
        super().__init__()
        self.BN_enable = BN_enable

        # encoder part
        # Using the pretrained ResNet-50 model directly from torchvision
        resnet = models.resnet50(pretrained=True)

        filters = [64, 256, 512, 1024, 2048]
        self.firstconv = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.firstbn = resnet.bn1
        self.firstrelu = resnet.relu
        self.firstmaxpool = resnet.maxpool
        self.encoder1 = resnet.layer1
        self.encoder2 = resnet.layer2
        self.encoder3 = resnet.layer3

        # decoder part
        self.center = DecoderBlock(in_channels=filters[3], mid_channels=filters[3]*4, out_channels=filters[3], BN_enable=self.BN_enable)
        self.decoder1 = DecoderBlock(in_channels=filters[3]+filters[2], mid_channels=filters[2]*4, out_channels=filters[2], BN_enable=self.BN_enable)
        self.decoder2 = DecoderBlock(in_channels=filters[2]+filters[1], mid_channels=filters[1]*4, out_channels=filters[1], BN_enable=self.BN_enable)
        self.decoder3 = DecoderBlock(in_channels=filters[1]+filters[0], mid_channels=filters[0]*4, out_channels=filters[0], BN_enable=self.BN_enable)

        if self.BN_enable:
            self.final = nn.Sequential(
                nn.Conv2d(in_channels=filters[0], out_channels=32, kernel_size=3, padding=1),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=False),
                nn.Conv2d(in_channels=32, out_channels=1, kernel_size=1),
                nn.Sigmoid()
            )
        else:
            self.final = nn.Sequential(
                nn.Conv2d(in_channels=filters[0], out_channels=32, kernel_size=3, padding=1),
                nn.ReLU(inplace=False),
                nn.Conv2d(in_channels=32, out_channels=1, kernel_size=1),
                nn.Sigmoid()
            )

    def forward(self, x):
        x = self.firstconv(x)
        x = self.firstbn(x)
        x = self.firstrelu(x)
        x_ = self.firstmaxpool(x)

        e1 = self.encoder1(x_)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)

        center = self.center(e3)

        d2 = self.decoder1(torch.cat([center, e2], dim=1))
        d3 = self.decoder2(torch.cat([d2, e1], dim=1))
        d4 = self.decoder3(torch.cat([d3, x], dim=1))

        return self.final(d4)

# Instantiate the model
model = Resnet_Unet(BN_enable=True)

# Optionally load the model weights if you saved them separately
# model.load_state_dict(torch.load('your_model_weights.pth'))


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 209MB/s]


In [ ]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch.nn.functional as F

# Load full dataset function (RGB)
def load_full_dataset(train_image_folder, train_mask_folder):
    images = []
    masks = []

    # List all image files in the image folder
    image_files = sorted(os.listdir(train_image_folder))
    mask_files = sorted(os.listdir(train_mask_folder))

    for img_file, mask_file in zip(image_files, mask_files):
        # Construct full file path
        img_path = os.path.join(train_image_folder, img_file)
        mask_path = os.path.join(train_mask_folder, mask_file)

        # Load the image as RGB
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # Read as RGB
        if img is None:
            print(f"Warning: Unable to read image {img_path}")
            continue
        img = cv2.resize(img, (256, 256))  # Resize to match the model input

        # Load the mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        if mask is None:
            print(f"Warning: Unable to read mask {mask_path}")
            continue
        mask = cv2.resize(mask, (256, 256))  # Resize to match the model input

        # Append to lists
        images.append(img)
        masks.append(mask.reshape((256, 256, 1)))  # Add channel dimension to masks

    return np.array(images), np.array(masks)

# Specify your folder paths
train_image_folder = '/content/drive/MyDrive/iirs/small_data/processed_tiles_of_tiff_delhi_ncr'
train_mask_folder = '/content/drive/MyDrive/iirs/small_data/processed_raster_mask_tiles'

# Load your entire dataset
all_images, all_masks = load_full_dataset(train_image_folder, train_mask_folder)

# Ensure the data is in the right shape
print(f"Loaded images shape: {all_images.shape}")  # Should be (99, 256, 256, 3)
print(f"Loaded masks shape: {all_masks.shape}")    # Should be (99, 256, 256, 1)

# Normalize images if necessary
all_images = all_images.astype('float32') / 255.0
all_masks = all_masks.astype('float32') / 255.0  # Ensure masks are also in the right format

# Check the normalized data shapes
print(f"Normalized images shape: {all_images.shape}")
print(f"Normalized masks shape: {all_masks.shape}")

# U-Net Model Definition
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels, upsample_mode='pixelshuffle', BN_enable=True):
        super().__init__()
        self.in_channels = in_channels
        self.mid_channels = mid_channels
        self.out_channels = out_channels
        self.upsample_mode = upsample_mode
        self.BN_enable = BN_enable

        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=mid_channels, kernel_size=3, stride=1, padding=1, bias=False)

        if self.BN_enable:
            self.norm1 = nn.BatchNorm2d(mid_channels)
        self.relu1 = nn.ReLU(inplace=False)
        self.relu2 = nn.ReLU(inplace=False)

        if self.upsample_mode == 'deconv':
            self.upsample = nn.ConvTranspose2d(in_channels=mid_channels, out_channels=out_channels,
                                                kernel_size=3, stride=2, padding=1, output_padding=1, bias=False)
        elif self.upsample_mode == 'pixelshuffle':
            self.upsample = nn.PixelShuffle(upscale_factor=2)
        if self.BN_enable:
            self.norm2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        if self.BN_enable:
            x = self.norm1(x)
        x = self.relu1(x)
        x = self.upsample(x)
        if self.BN_enable:
            x = self.norm2(x)
        x = self.relu2(x)
        return x

class Resnet_Unet(nn.Module):
    def __init__(self, BN_enable=True, resnet_pretrain=False):
        super().__init__()
        self.BN_enable = BN_enable
        resnet = models.resnet50(pretrained=resnet_pretrain)
        filters = [64, 256, 512, 1024, 2048]
        self.firstconv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)  # Changed to 3 channels
        self.firstbn = resnet.bn1
        self.firstrelu = resnet.relu
        self.firstmaxpool = resnet.maxpool
        self.encoder1 = resnet.layer1
        self.encoder2 = resnet.layer2
        self.encoder3 = resnet.layer3

        # Decoder parts
        self.center = DecoderBlock(in_channels=filters[3], mid_channels=filters[3]*4, out_channels=filters[3], BN_enable=self.BN_enable)
        self.decoder1 = DecoderBlock(in_channels=filters[3]+filters[2], mid_channels=filters[2]*4, out_channels=filters[2], BN_enable=self.BN_enable)
        self.decoder2 = DecoderBlock(in_channels=filters[2]+filters[1], mid_channels=filters[1]*4, out_channels=filters[1], BN_enable=self.BN_enable)
        self.decoder3 = DecoderBlock(in_channels=filters[1]+filters[0], mid_channels=filters[0]*4, out_channels=filters[0], BN_enable=self.BN_enable)

        if self.BN_enable:
            self.final = nn.Sequential(
                nn.Conv2d(in_channels=filters[0], out_channels=32, kernel_size=3, padding=1),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=False),
                nn.Conv2d(in_channels=32, out_channels=1, kernel_size=1),
                nn.Sigmoid()
            )
        else:
            self.final = nn.Sequential(
                nn.Conv2d(in_channels=filters[0], out_channels=32, kernel_size=3, padding=1),
                nn.ReLU(inplace=False),
                nn.Conv2d(in_channels=32, out_channels=1, kernel_size=1),
                nn.Sigmoid()
            )

    def forward(self, x):
        x = self.firstconv(x)
        x = self.firstbn(x)
        x = self.firstrelu(x)
        x_ = self.firstmaxpool(x)

        e1 = self.encoder1(x_)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)

        center = self.center(e3)

        d2 = self.decoder1(torch.cat([center, e2], dim=1))
        d3 = self.decoder2(torch.cat([d2, e1], dim=1))
        d4 = self.decoder3(torch.cat([d3, x], dim=1))

        return self.final(d4)

# Function to calculate accuracy
def calculate_accuracy(outputs, masks):
    predicted = (outputs > 0.5).float()  # Apply a threshold of 0.5
    correct = (predicted == masks).float()  # Compare predicted with true masks
    accuracy = correct.sum() / correct.numel()  # Calculate accuracy
    return accuracy.item()

# Instantiate the model
model = Resnet_Unet(BN_enable=True)

# Optionally load model weights if saved separately
# model.load_state_dict(torch.load('path_to_weights.pth'))

# Set up training parameters
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss

# Prepare dataset for training
train_images = torch.from_numpy(all_images).permute(0, 3, 1, 2)  # Change shape to (N, C, H, W)
train_masks = torch.from_numpy(all_masks)

# Split the dataset into training and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(
    train_images, train_masks, test_size=0.2, random_state=42
)

# Create DataLoader
train_dataset = TensorDataset(train_images, train_masks)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()  # Set model to training mode
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    total_batches = 0

    for imgs, masks in train_loader:
        optimizer.zero_grad()
        outputs = model(imgs.float())  # Forward pass

        # Reshape masks to match the output shape (N, C, H, W)
        masks = masks.permute(0, 3, 1, 2)  # Reshape

        loss = criterion(outputs, masks.float())  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize weights

        # Calculate accuracy
        accuracy = calculate_accuracy(outputs, masks.float())

        epoch_loss += loss.item()
        epoch_accuracy += accuracy
        total_batches += 1

    # Average loss and accuracy for the epoch
    avg_loss = epoch_loss / total_batches
    avg_accuracy = epoch_accuracy / total_batches

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.4f}")

print("Training complete.")


Loaded images shape: (99, 256, 256, 3)
Loaded masks shape: (99, 256, 256, 1)
Normalized images shape: (99, 256, 256, 3)
Normalized masks shape: (99, 256, 256, 1)
Epoch [1/50], Loss: 0.6540, Accuracy: 0.7669
Epoch [2/50], Loss: 0.5378, Accuracy: 0.9278
Epoch [3/50], Loss: 0.4622, Accuracy: 0.9260
Epoch [4/50], Loss: 0.4185, Accuracy: 0.9295
Epoch [5/50], Loss: 0.3959, Accuracy: 0.9307
Epoch [6/50], Loss: 0.3753, Accuracy: 0.9345
Epoch [7/50], Loss: 0.3563, Accuracy: 0.9328
Epoch [8/50], Loss: 0.3419, Accuracy: 0.9327
Epoch [9/50], Loss: 0.3271, Accuracy: 0.9337
Epoch [10/50], Loss: 0.3168, Accuracy: 0.9343
Epoch [11/50], Loss: 0.3044, Accuracy: 0.9338
Epoch [12/50], Loss: 0.2936, Accuracy: 0.9357
Epoch [13/50], Loss: 0.2833, Accuracy: 0.9336
Epoch [14/50], Loss: 0.2760, Accuracy: 0.9330
Epoch [15/50], Loss: 0.2677, Accuracy: 0.9333
Epoch [16/50], Loss: 0.2583, Accuracy: 0.9367
Epoch [17/50], Loss: 0.2492, Accuracy: 0.9351
Epoch [18/50], Loss: 0.2410, Accuracy: 0.9325
Epoch [19/50], Loss

GIVING ONLY LOSS(GOOD FOR JUGAD)

In [ ]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch.nn.functional as F

# Load full dataset function (RGB)
def load_full_dataset(train_image_folder, train_mask_folder):
    images = []
    masks = []

    # List all image files in the image folder
    image_files = sorted(os.listdir(train_image_folder))
    mask_files = sorted(os.listdir(train_mask_folder))

    for img_file, mask_file in zip(image_files, mask_files):
        # Construct full file path
        img_path = os.path.join(train_image_folder, img_file)
        mask_path = os.path.join(train_mask_folder, mask_file)

        # Load the image as RGB
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # Read as RGB
        if img is None:
            print(f"Warning: Unable to read image {img_path}")
            continue
        img = cv2.resize(img, (256, 256))  # Resize to match the model input

        # Load the mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        if mask is None:
            print(f"Warning: Unable to read mask {mask_path}")
            continue
        mask = cv2.resize(mask, (256, 256))  # Resize to match the model input

        # Append to lists
        images.append(img)
        masks.append(mask.reshape((256, 256, 1)))  # Add channel dimension to masks

    return np.array(images), np.array(masks)

# Specify your folder paths
train_image_folder = '/content/drive/MyDrive/iirs/small_data/processed_tiles_of_tiff_delhi_ncr'
train_mask_folder = '/content/drive/MyDrive/iirs/small_data/processed_raster_mask_tiles'

# Load your entire dataset
all_images, all_masks = load_full_dataset(train_image_folder, train_mask_folder)

# Ensure the data is in the right shape
print(f"Loaded images shape: {all_images.shape}")  # Should be (99, 256, 256, 3)
print(f"Loaded masks shape: {all_masks.shape}")    # Should be (99, 256, 256, 1)

# Normalize images if necessary
all_images = all_images.astype('float32') / 255.0
all_masks = all_masks.astype('float32') / 255.0  # Ensure masks are also in the right format

# Check the normalized data shapes
print(f"Normalized images shape: {all_images.shape}")
print(f"Normalized masks shape: {all_masks.shape}")

# U-Net Model Definition
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels, upsample_mode='pixelshuffle', BN_enable=True):
        super().__init__()
        self.in_channels = in_channels
        self.mid_channels = mid_channels
        self.out_channels = out_channels
        self.upsample_mode = upsample_mode
        self.BN_enable = BN_enable

        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=mid_channels, kernel_size=3, stride=1, padding=1, bias=False)

        if self.BN_enable:
            self.norm1 = nn.BatchNorm2d(mid_channels)
        self.relu1 = nn.ReLU(inplace=False)
        self.relu2 = nn.ReLU(inplace=False)

        if self.upsample_mode == 'deconv':
            self.upsample = nn.ConvTranspose2d(in_channels=mid_channels, out_channels=out_channels,
                                                kernel_size=3, stride=2, padding=1, output_padding=1, bias=False)
        elif self.upsample_mode == 'pixelshuffle':
            self.upsample = nn.PixelShuffle(upscale_factor=2)
        if self.BN_enable:
            self.norm2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        if self.BN_enable:
            x = self.norm1(x)
        x = self.relu1(x)
        x = self.upsample(x)
        if self.BN_enable:
            x = self.norm2(x)
        x = self.relu2(x)
        return x

class Resnet_Unet(nn.Module):
    def __init__(self, BN_enable=True, resnet_pretrain=False):
        super().__init__()
        self.BN_enable = BN_enable
        resnet = models.resnet50(pretrained=resnet_pretrain)
        filters = [64, 256, 512, 1024, 2048]
        self.firstconv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)  # Changed to 3 channels
        self.firstbn = resnet.bn1
        self.firstrelu = resnet.relu
        self.firstmaxpool = resnet.maxpool
        self.encoder1 = resnet.layer1
        self.encoder2 = resnet.layer2
        self.encoder3 = resnet.layer3

        # Decoder parts
        self.center = DecoderBlock(in_channels=filters[3], mid_channels=filters[3]*4, out_channels=filters[3], BN_enable=self.BN_enable)
        self.decoder1 = DecoderBlock(in_channels=filters[3]+filters[2], mid_channels=filters[2]*4, out_channels=filters[2], BN_enable=self.BN_enable)
        self.decoder2 = DecoderBlock(in_channels=filters[2]+filters[1], mid_channels=filters[1]*4, out_channels=filters[1], BN_enable=self.BN_enable)
        self.decoder3 = DecoderBlock(in_channels=filters[1]+filters[0], mid_channels=filters[0]*4, out_channels=filters[0], BN_enable=self.BN_enable)

        if self.BN_enable:
            self.final = nn.Sequential(
                nn.Conv2d(in_channels=filters[0], out_channels=32, kernel_size=3, padding=1),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=False),
                nn.Conv2d(in_channels=32, out_channels=1, kernel_size=1),
                nn.Sigmoid()
            )
        else:
            self.final = nn.Sequential(
                nn.Conv2d(in_channels=filters[0], out_channels=32, kernel_size=3, padding=1),
                nn.ReLU(inplace=False),
                nn.Conv2d(in_channels=32, out_channels=1, kernel_size=1),
                nn.Sigmoid()
            )

    def forward(self, x):
        x = self.firstconv(x)
        x = self.firstbn(x)
        x = self.firstrelu(x)
        x_ = self.firstmaxpool(x)

        e1 = self.encoder1(x_)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)

        center = self.center(e3)

        d2 = self.decoder1(torch.cat([center, e2], dim=1))
        d3 = self.decoder2(torch.cat([d2, e1], dim=1))
        d4 = self.decoder3(torch.cat([d3, x], dim=1))

        return self.final(d4)

# Instantiate the model
model = Resnet_Unet(BN_enable=True)

# Optionally load model weights if saved separately
# model.load_state_dict(torch.load('path_to_weights.pth'))

# Set up training parameters
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss

# Prepare dataset for training
train_images = torch.from_numpy(all_images).permute(0, 3, 1, 2)  # Change shape to (N, C, H, W)
train_masks = torch.from_numpy(all_masks)

# Split the dataset into training and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(
    train_images, train_masks, test_size=0.2, random_state=42
)

# Create DataLoader
train_dataset = TensorDataset(train_images, train_masks)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

epochs = 50
for epoch in range(epochs):
    model.train()  # Set model to training mode
    for imgs, masks in train_loader:
        optimizer.zero_grad()
        outputs = model(imgs.float())  # Forward pass
        # Reshape masks to match the output shape (N, C, H, W)
        masks = masks.permute(0, 3, 1, 2)  # Reshape
        loss = criterion(outputs, masks.float())  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize weights

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

print("Training complete.")
